In [99]:
import pandas as pd
import numpy as np
import chardet
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import string
import utils.data_processing_utils as util

In [100]:


def process_data(data_frame,text_label,y_label,remove_text_with_substrings = None, remove_characters_from_text = None, remove_column_indexs =None, drop_na = False, remove_word_count_outlier=False,remove_word_length_outlier=False, remove_white_space = False):
    
    #Remove columns based off column index
    if(remove_column_indexs != None):
       data_frame.drop(data_frame.columns[remove_column_indexs],axis=1,inplace=True)

    #Remove null values
    if(drop_na):
       data_frame.dropna()

    #Remove white space
    if(remove_white_space):
        data_frame[text_label] = util.remove_block_of_white_space(data_frame[text_label])

    #Remove rows with substring from the data   
    if(remove_text_with_substrings != None):
        for substring in remove_text_with_substrings:
            data_frame = util.remove_text_with_substring(data_frame,data_frame[text_label],substring)

    #Remove characters within the rows of the data  
    if(remove_characters_from_text != None):
        data_frame[text_label] = util.remove_character_from_text(data_frame[text_label],remove_characters_from_text)


    #Remove outliers
    if(remove_word_count_outlier == True or remove_word_length_outlier == True):
        # Create word count and character count lists
        word_count, char_count, ave_length =  util.getTextMetaInformation(data_frame,text_label)
        word_count_outliers = []
        word_length_outliers = []

        if(remove_word_count_outlier):
            #Get text indexes with short word count 
            short_word_count_outliers = util.get_outlier_indexes(word_count, 5, "less")
            #Get text indexes with long word count 
            long_word_count_outliers = util.get_outlier_indexes(word_count, 35, "greater")
            #Outliers for word count
            word_count_outliers = np.append(short_word_count_outliers, long_word_count_outliers)
        if(remove_word_length_outlier):
            # Get long word length outliers
            long_word_length_outliers = util.get_outlier_indexes(ave_length, 23, "greater")
            # Get short word length outliers
            short_word_length_outliers = util.get_outlier_indexes(ave_length, 3.5, "less")
            #Outliers for word length
            word_length_outliers = np.append(short_word_length_outliers, long_word_length_outliers)

    if(remove_word_count_outlier and remove_word_length_outlier ):
        outliers = np.append(word_count_outliers,word_length_outliers)       
        data_frame = data_frame.drop(data_frame.index[outliers])

    elif(remove_word_count_outlier and not remove_word_length_outlier):
        outliers = word_count_outliers      
        data_frame = data_frame.drop(data_frame.index[outliers])   

    elif(not remove_word_count_outlier and remove_word_length_outlier):
        outliers = word_length_outliers      
        data_frame = data_frame.drop(data_frame.index[outliers])   

    return data_frame


In [101]:
#Parameters
author_label_name = "username"
text_label = "raw_text"
file_name = "authorship_dataset.csv"
remove_characters = ['ï', 'é', 'ñ', 'è', 'ö', 'æ', 'ô', 'â', 'á', 'à', 'ê', 'ë']
remove_column_indexes =[0,1]
remove_substrings_from_text = ["RT"]
drop_na = True
remove_word_count_outlier = False
remove_word_length_outlier = False
remove_white_space = True


In [102]:
data = pd.read_csv(file_name)
len(data)

63019

In [103]:
df = process_data(data,text_label,author_label_name,remove_substrings_from_text,remove_characters,
remove_column_indexes,drop_na,remove_word_count_outlier,remove_word_length_outlier,remove_white_space)
len(df)

63019


C:\Users\leobl\AppData\Local\Temp\ipykernel_14692\456196336.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[text_label] = util.remove_character_from_text(data_frame[text_label],remove_characters_from_text)


27873

In [104]:
#save to excel
df.to_csv("processed_data.csv")